In [ ]:
!wget -q --show-progress -O training.tar.zip "https://www.dropbox.com/scl/fi/rihfngx4ju5pzjzjj7u9z/lesson6.tar.zip?rlkey=rct9a9bo8euqgshrk8wiq2orh&dl=1"

!tar -xzvf training.tar.zip

!tar -xzvf lesson6.tar



training.tar.zip    100%[===================>]   3.93K  --.-KB/s    in 0s      
sample.log
._training.txt
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.metadata:kMDItemTextContentLanguage'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.metadata:kMDItemKeyphraseVersion'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.metadata:kMDItemKeyphraseLabels'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.metadata:kMDItemKeyphraseConfidences'
training.txt
tar (child): lesson6.tar: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
!pip install python-dotenv


In [ ]:
import os
import sys
from dotenv import load_dotenv, find_dotenv

class Utils:
  def __init__(self):
    pass

  def create_dlai_index_name(self, index_name):
    openai_key = "enter api key"

    return f'{index_name}-{openai_key[-36:].lower().replace("_", "-")}'

  def is_colab(self):
    return 'google.colab' in sys.modules

  def get_openai_api_key(self):
    _ = load_dotenv(find_dotenv())
    return os.getenv("OPENAI_API_KEY")

  def get_pinecone_api_key(self):
    _ = load_dotenv(find_dotenv())
    return os.getenv("PINECONE_API_KEY")


In [ ]:
OPENAI_API_KEY = "define open api key"

# New Section

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer, InputExample, losses, models, util
from torch.utils.data import DataLoader
from torch import nn
from tqdm.auto import tqdm
import torch
import time
import torch
import os

In [ ]:
!pip install DLAIUtils

ERROR: Could not find a version that satisfies the requirement DLAIUtils (from versions: none)
ERROR: No matching distribution found for DLAIUtils


In [ ]:
!pip install Pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!head -5 sample.log

Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0]
Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, code 0, by access-group "acl_dmz" [0xe3aab522, 0x0]
Apr 15 2014 09:34:34 EDT: %ASA-session-5-106100: access-list acl_in permitted tcp inside/10.1.2.16(2241) -> outside/192.0.0.89(2000) hit-cnt 1 first hit [0x71a87d94, 0x0]
Apr 24 2013 16:00:28 INT-FW01 : %ASA-6-106100: access-list inside denied udp inside/172.29.2.101(1039) -> outside/192.0.2.10(53) hit-cnt 1 first hit [0xd820e56a, 0x0]
Apr 24 2013 16:00:27 INT-FW01 : %ASA-6-106100: access-list inside permitted udp inside/172.29.2.3(1065) -> outside/192.0.2.57(53) hit-cnt 144 300-second interval [0xe982c7a4, 0x0]


In [ ]:
!head -5 training.txt

Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0] ^ Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0] ^ 1.0
Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, code 0, by access-group "acl_dmz" [0xe3aab522, 0x0] ^ Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0] ^ 0.9
Apr 15 2014 09:34:34 EDT: %ASA-session-5-106100: access-list acl_in permitted tcp inside/10.1.2.16(2241) -> outside/192.0.0.89(2000) hit-cnt 1 first hit [0x71a87d94, 0x0] ^ Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0] ^ 0.8
Apr 24 2013 16:00:28 INT-FW01 : %ASA-6-106100: access-list inside denied udp inside/172.29.2.1

In [ ]:
utils = Utils()

In [ ]:
PINECONE_API_KEY = "write pincone api key"

In [ ]:
INDEX_NAME = utils.create_dlai_index_name('dl-ai')

pinecone = Pinecone(api_key=PINECONE_API_KEY)

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)
pinecone.create_index(name=INDEX_NAME, dimension=256, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))
index = pinecone.Index(INDEX_NAME)

In [ ]:
index

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=768)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model], device=device)
device

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

'cpu'

In [ ]:
train_examples = []
with open('./training.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if line:
            a, b, label = line.split('^')
            train_examples.append(InputExample(texts=[a, b], label=float(label)))

#Define dataset, the dataloader and the training loss
warmup_steps=100
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
import pickle
load_pretrained_model = False
if load_pretrained_model:
    trained_model_file = open('./pretrained_model', 'rb')
    db = pickle.load(trained_model_file)
    trained_model_file.close()
else:
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=16, warmup_steps=100)

samples = []
with open('sample.log', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if line:
            #emb = model.encode([line])
            samples.append(line)

Epoch:   0%|          | 0/16 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
embeddings = model.encode(samples)

# Prepare data for Pinecone
data = [{'id': str(i), 'values': embeddings[i].tolist(), 'metadata': {'log': samples[i]}} for i in range(len(samples))]

# Upsert data to Pinecone
index.upsert(vectors=data)

{'upserted_count': 90}

In [ ]:
embeddings[0]

array([-1.70149297e-01, -1.37123689e-01,  2.76358455e-01,  2.71750778e-01,
       -2.16508493e-01,  3.01490664e-01, -2.95678768e-02, -3.69758494e-02,
        1.81712553e-01, -1.51978731e-01, -8.37911516e-02, -3.00729007e-01,
       -3.89178365e-01, -1.71394483e-03, -3.49937111e-01, -4.94637191e-02,
        1.37571782e-01, -1.91429839e-01,  3.62390369e-01, -1.22994289e-01,
       -2.90628046e-01,  2.02595107e-02,  1.59556627e-01,  1.66452751e-01,
       -6.58692559e-03, -5.56719303e-02,  2.55123615e-01, -1.70365319e-01,
       -2.83915251e-01, -1.36313960e-01, -5.28861582e-02,  1.06607758e-01,
       -4.04189304e-02,  2.58255154e-01, -1.45379543e-01,  1.65624470e-01,
       -2.52481341e-01, -1.31066471e-01, -2.48084158e-01,  3.72325778e-01,
       -2.99032360e-01, -7.27533922e-02, -1.67788997e-01,  2.48126134e-01,
       -2.38562152e-01,  6.50263503e-02,  6.55982792e-02,  1.88559100e-01,
       -2.00152416e-02, -2.47024313e-01, -3.02949667e-01, -3.03418994e-01,
        2.22716048e-01,  

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


AttributeError: module 'openai' has no attribute 'error'

In [ ]:
from openai import OpenAI
import numpy as np

# Initialize OpenAI
openai_client = OpenAI(api_key = "write open api key")

def get_similar_logs(query, top_k=100):
    #query_embedding = model.encode([query])
    response = index.query(vector = embeddings[0].tolist(), top_k=top_k, include_metadata=True)
    return response['matches']

def generate_response(query):
    similar_logs = get_similar_logs(query, top_k=5)
    context = "\n".join([match['metadata']['log'] for match in similar_logs])

    response = openai_client.completions.create(
        model="gpt-3.5-turbo-instruct-0914",
        prompt=f"Context: {context}\n\nQuery: {query}\n\nResponse:",
        max_tokens=1024,
        temperature=0.7,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    if response:
        # Check if response has 'choices' attribute
        if 'choices' in response:
            return response['choices'][0]['text'].strip()
        else:
            return response
    else:
        return "Failed to get response from OpenAI API."

# Example usage
query = "Find all the anomalies in the log, tell why its an anomaly? "
response = generate_response(query)
print(response)


Completion(id='cmpl-9PfQeD7UZGGuWpbVzPUnhEsfPUELU', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n1. The first anomaly is the source IP address in the first two log entries. The source IP address is from a network range (10.1.2.30) that is designated for the DMZ, but the destination IP address is outside of the network (192.0.0.8). This could indicate an attempt at unauthorized access from the DMZ to an external network.\n\n2. The third and fourth log entries also have a mismatch between the source and destination IP addresses. The source IP address is from an external network (192.0.2.126) and the destination IP address is from the internal network (10.0.0.132). This could indicate an attempted attack or unauthorized access from an external network to an internal network.\n\n3. The last log entry also has a mismatch between the source and destination IP addresses. The source IP address is from an external network (192.0.2.95) and the destination IP ad

In [ ]:
response.choices[0].text

'\n1. The first anomaly is the source IP address in the first two log entries. The source IP address is from a network range (10.1.2.30) that is designated for the DMZ, but the destination IP address is outside of the network (192.0.0.8). This could indicate an attempt at unauthorized access from the DMZ to an external network.\n\n2. The third and fourth log entries also have a mismatch between the source and destination IP addresses. The source IP address is from an external network (192.0.2.126) and the destination IP address is from the internal network (10.0.0.132). This could indicate an attempted attack or unauthorized access from an external network to an internal network.\n\n3. The last log entry also has a mismatch between the source and destination IP addresses. The source IP address is from an external network (192.0.2.95) and the destination IP address is from a different internal network (10.32.112.125). This could indicate a potential security breach or attempted attack f

In [ ]:
samples[0]

'Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0]'

Improved Query

In [ ]:
import openai
import numpy as np
import pinecone
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

# Initialize OpenAI

# Create or connect to the Pinecone index


def get_log_from_query(query):
    # Extract log entry from the query
    log_start = query.find("for the log ")
    log_end = query.find("find anomalies")
    if log_start != -1 and log_end != -1:
        log_entry = query[log_start + len("for the log "):log_end].strip()
        return log_entry
    return None

def get_similar_logs(log_entry, top_k=100):
    # Encode the log entry
    query_embedding = model.encode([log_entry])
    response = index.query(vector=query_embedding[0].tolist(), top_k=top_k, include_metadata=True)
    return response['matches']

def generate_response(query):
    log_entry = get_log_from_query(query)
    if not log_entry:
        return "Failed to extract log entry from the query."

    similar_logs = get_similar_logs(log_entry, top_k=5)
    context = "\n".join([match['metadata']['log'] for match in similar_logs])
    print(context)
    # Use the remaining part of the query for generating the response
    remaining_query = query[query.find("find anomalies"):].strip()
    print(remaining_query)


    response =  openai_client.completions.create(
               model="gpt-3.5-turbo-instruct-0914",
        prompt=f"Context: {context}\n\nQuery: {query}\n\nResponse:",
        max_tokens=2048,
        temperature=0.7,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
            )




    return response

# Example usage
query = "for the log Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group \"acl_dmz\" [0xe3aab522, 0x0] find anomalies"
response = generate_response(query)
print(response.choices[0].text)


Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0]
Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, code 0, by access-group "acl_dmz" [0xe3aab522, 0x0]
Apr 30 2013 09:23:41: %ASA-4-106023: Deny tcp src outside:192.0.2.126/53638 dst inside:10.0.0.132/8111 by access-group "acl_out" [0x71761f18, 0x0]
Apr 30 2013 09:23:40: %ASA-4-106023: Deny tcp src outside:192.0.2.126/53638 dst inside:10.0.0.132/8111 by access-group "acl_out" [0x71761f18, 0x0]
Sep 12 2014 06:53:01 GIFRCHN01 : %ASA-4-106023: Deny tcp src outside:192.0.2.95/24069 dst inside:10.32.112.125/25 by access-group "PERMIT_IN" [0x0, 0x0]"
find anomalies


1. The log shows that access from the DMZ network (10.1.2.30) to the outside network (192.0.0.8) on port 53 was denied by the access-group "acl_dmz". This could indicate a potential security issue as port 53 is commonly used for DNS traffic 